In [42]:
import requests
import pandas as pd

def getProtocols():
    resp = requests.get('https://api.llama.fi/protocols')
    return resp.json()

protocols = getProtocols()
df = pd.json_normalize(protocols)

In [43]:
def getProtocolTVLHistorical(protocol_slug):
    url = f'https://api.llama.fi/protocol/{protocol_slug}'
    resp = requests.get(url)
    if resp.status_code == 200:
        resp = resp.json()
        # print(list(resp.keys()))
        # print(type(resp['tvl']))
        # print(resp['tvl'][0])  # 
        temp_dates, temp_tvls = [], []
        for dict in resp['tvl']:
            temp_dates.append(dict['date'])
            temp_tvls.append(dict['totalLiquidityUSD'])
        return temp_dates, temp_tvls
    
def processTVLData(tdates, ttvls):
    tdf = pd.DataFrame(data={'timestamp': tdates, 'tvls': ttvls})
    tdf['dates'] = tdf['timestamp'].values.astype(dtype='datetime64[s]')
    tdf['dates'] = pd.to_datetime(tdf['dates']).dt.date
    tdf['7dma'] = tdf['tvls'].rolling(window=7, min_periods=1).mean()
    tdf['7dma_%'] = tdf['7dma'].pct_change(periods=1).fillna(0)
    tdf['1mma'] = tdf['tvls'].rolling(window=30, min_periods=1).mean()
    tdf['1mma_%'] = tdf['1mma'].pct_change(periods=1).fillna(0)
    sevenday = tdf['7dma_%'].values.tolist()[-1]
    thirtyday = tdf['1mma_%'].values.tolist()[-1]
    return sevenday, thirtyday


min_tvl = 10_000_000
print(f'Min TVL: {min_tvl}')
df = df[df['tvl'] >= min_tvl]
print(f'Protocols Tracked: {len(df)}')
# slugs = df['slug'].values.tolist()
# print(slugs[0])
# sevendayma, thirtydayma = [], []
# for slug in slugs:
#     dates, tvls = getProtocolTVLHistorical(slug)
#     tempa, tempb = processTVLData(dates, tvls)
#     sevendayma.append(tempa)
#     thirtydayma.append(tempb)
# df['7dma_%'] = sevendayma
# df['1mma_%'] = thirtydayma

Min TVL: 10000000
Protocols Tracked: 393


In [44]:
# volume stats
url = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
resp = requests.get(url)
resp = resp.json()


In [75]:
url = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
resp = requests.get(url)
resp = resp.json()
# print(df)
# print(resp['protocols'][0])
df_vol = pd.json_normalize(resp['protocols'])
# REname defillamJoin on 
df_vol = df_vol.rename(columns={'defillamaId': 'id'})
df_vol = df_vol[['id', 'change_7d', 'change_1m']]
# print(df_vol)
print(len(df_vol))

url = 'https://api.llama.fi/overview/options?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyPremiumVolume'
resp = requests.get(url).json()
# print(resp)
df_vol2 = pd.json_normalize(resp['protocols'])
df_vol2 = df_vol2.rename(columns={'defillamaId': 'id'})
df_vol2 = df_vol2[['id', 'change_7d', 'change_1m']]
# print(df_vol2)
print(len(df_vol2))
df_vol = pd.concat([df_vol, df_vol2]).reset_index(drop=True)
df_vol['id'].astype(int)
print(df_vol)


305
4
       id  change_7d  change_1m
0    2116     -30.21      90.84
1    2345      23.04      -5.02
2    2283     -55.23     -72.77
3     951     -63.44     -60.61
4    2695     -61.77     -74.42
..    ...        ...        ...
304  2198     -52.84     -36.88
305   128     -99.02     -98.30
306   503      42.96     -47.49
307   381      48.29     -62.22
308   534     -48.18      32.63

[309 rows x 3 columns]


In [95]:
df = df[['id', 'name', 'address', 'symbol']]
print(len(df))
df['id'].astype(int)
# print(df.info())
df['7dma_%'] = "7"
df['1mma_%'] = "30"
# print(df_vol.info())
df_vol['id'].astype(int)
print(len(df_vol))
df = df.merge(df_vol, how='left', on='id')
print(df['change_7d'].count())


393
309
81


In [4]:
a = []
if a:
    print('a')
else:
    print('false')

false


In [4]:
import pandas as pd

df = pd.read_csv('variant_db.csv')
print(len(df))
# print(len(df))
# print(df['address'].count())
# df['address'] = df['address'].fillna('No')
# df2 = df[~df["address"].str.contains(':')]
# df2 = df2[df2["address"] != 'No'].reset_index(drop=True)
# print(len(df2))
# print(df2)

hdc = df['address'].values.tolist()

wallet_list = []
# print(len(hdc))
for i, address in enumerate(hdc):
    if address == address and (":" not in address):
        # print(f'searching: {address}')
        # Get Holder Count Data (7d & 1m change)
        wallet_list.append(address)
        
wdf = pd.DataFrame(data={'wallet': wallet_list})
wdf.to_csv('testaddresses.csv', index=False)
print(wdf)

416
                                         wallet
0    0x5a98fcbea516cf06857215779fd812ca3bef1b32
1    0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
2    0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0
3    0x2260fac5e5542a773aa44fbcfedf7c193bc2c599
4    0xD533a949740bb3306d119CC777fa900bA034cd52
..                                          ...
167  0xde30da39c46104798bb5aa3fe8b9e0e1f348163f
168  0xc18360217d8f7ab5e7c516566761ea12ce7f9d72
169  0x6810e776880c02933d47db1b9fc05908e5386b96
170  0x514910771af9ca656af840dff83e8264ecf986ca
171  0x4d224452801aced8b2f0aebe155379bb5d594381

[172 rows x 1 columns]
